In [58]:
import pandas as pd
import re

# Read in the CSV file 
applicants_2023 = pd.read_csv('C:/Users/ksultana/Videos/Internship/2023Applicants.csv')
print(applicants_2023.head(100))
print(applicants_2023.info())

    record_id  selection_type age  gender_id underrepresented ethnicity_other  \
0           1               0  20          1                7             NaN   
1           2               0  21          2                7             NaN   
2           3               0  19          2                8             NaN   
3           4               2  21          2                8             NaN   
4           5               0  19          1             5,10             NaN   
..        ...             ...  ..        ...              ...             ...   
95         96               0  20          2                7             NaN   
96         97               1  20          1             8,10             NaN   
97         98               0  21          1                3             NaN   
98         99               0  19          1                3             NaN   
99        100               0  20          1                8             NaN   

   disadvantage  local     

In [59]:
def extract_number_or_decimal(value):
    if isinstance(value, (str, bytes)):
        # Extract numerical values and decimals using regular expression
        match = re.search(r'\d+\.\d+|\d+', value)
        if match:
            return match.group()
    return None

# Drop rows where 'age' column has value '#NUM!'
applicants_2023 = applicants_2023[applicants_2023['age'] != '#NUM!']

# Convert 'age' column to integer type after dropping rows with '#NUM!'
applicants_2023['age'] = applicants_2023['age'].astype(int)

# Drop rows where 'grade_average' column has value 'N/a'
applicants_2023 = applicants_2023[applicants_2023['grade_average'] != 'N/a']

# Extract numerical values or decimals from 'grade_average' column
applicants_2023['grade_average'] = applicants_2023['grade_average'].apply(extract_number_or_decimal)

# Convert to float
applicants_2023['grade_average'] = applicants_2023['grade_average'].astype(float)

# Drop rows with empty values in 'grade_average' column
applicants_2023 = applicants_2023.dropna(subset=['grade_average'])

# Fill empty values in 'local' column with 0
applicants_2023['local'] = applicants_2023['local'].fillna(0)

# Dropping unnecessary columns from the applicants_2023 DataFrame
applicants_2023 = applicants_2023.drop(columns=['record_id','ethnicity_other', 'state', 'zipcode', 'city', 'sigsub',
                                                'covid_question', 'institution','graduation_date',
                                                'application_submission_form_complete'])

# Drop rows with missing values only for the 'citizenship_2' column
applicants_2023.dropna(subset=['citizenship_2'], inplace=True)

# Drop rows with missing values only for the 'project_one_5' column
applicants_2023.dropna(subset=['project_one_5'], inplace=True)

# Converting specific columns to categorical type
applicants_2023[['selection_type', 'gender_id', 'local', 'citizenship','citizenship_2','classification','research_opportunities','available','summer_classes',
                'concentration_choice1','concentration_choice2','concentration_choice3','research_experience',
                 'fp_choice_1','fp_choice_2','fp_choice_3','project_one','project_one_2','project_one_3',
                 'project_one_4','project_one_5']]= applicants_2023[['selection_type', 'gender_id', 'local','citizenship','citizenship_2','classification',
'research_opportunities','available','summer_classes',
'concentration_choice1','concentration_choice2','concentration_choice3','research_experience','fp_choice_1','fp_choice_2','fp_choice_3','project_one','project_one_2'
                   ,'project_one_3','project_one_4','project_one_5']].astype('category')


print(applicants_2023.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 556 entries, 0 to 568
Data columns (total 27 columns):
 #   Column                  Non-Null Count  Dtype   
---  ------                  --------------  -----   
 0   selection_type          556 non-null    category
 1   age                     556 non-null    int32   
 2   gender_id               556 non-null    category
 3   underrepresented        538 non-null    object  
 4   disadvantage            538 non-null    object  
 5   local                   556 non-null    category
 6   citizenship             556 non-null    category
 7   citizenship_2           556 non-null    category
 8   classification          556 non-null    category
 9   undergrad_major         541 non-null    object  
 10  minor                   235 non-null    object  
 11  grade_average           556 non-null    float64 
 12  research_opportunities  556 non-null    category
 13  available               556 non-null    category
 14  summer_classes          55

In [60]:
# Split the multiple values in the 'underrepresented' column and expand into separate columns
split_columns = applicants_2023['underrepresented'].str.split(',', expand=True)

# Get unique values from all split columns
unique_values = split_columns.unstack().dropna().unique()

# Create new columns for each unique value
for value in unique_values:
    applicants_2023[f'underrepresented_{value}'] = split_columns.apply(lambda row: 1 if str(value) in row.values else 0, axis=1)
    
# List of newly created columns representing different categories extracted from the 'underrepresented' column
columns_to_convert = ['underrepresented_2', 'underrepresented_3', 'underrepresented_4','underrepresented_5','underrepresented_6',
                     'underrepresented_7','underrepresented_8','underrepresented_9','underrepresented_10', 'underrepresented_11'] 

# Iterate through columns_to_convert list and set the data type to 'category'
for column in columns_to_convert:
    applicants_2023[column] = applicants_2023[column].astype('category')

# Drop the original 'underrepresented' column
applicants_2023 = applicants_2023.drop(columns=['underrepresented'])

# Print the data types to confirm the changes
print(applicants_2023.dtypes)

print(applicants_2023.head(100))

selection_type            category
age                          int32
gender_id                 category
disadvantage                object
local                     category
citizenship               category
citizenship_2             category
classification            category
undergrad_major             object
minor                       object
grade_average              float64
research_opportunities    category
available                 category
summer_classes            category
concentration_choice1     category
concentration_choice2     category
concentration_choice3     category
research_experience       category
fp_choice_1               category
fp_choice_2               category
fp_choice_3               category
project_one               category
project_one_2             category
project_one_3             category
project_one_4             category
project_one_5             category
underrepresented_7        category
underrepresented_5        category
underrepresented_3  

In [61]:
# Split the multiple values in the 'underrepresented' column and expand into separate columns
split_columns = applicants_2023['disadvantage'].str.split(',', expand=True)

# Get unique values from all split columns
unique_values = split_columns.unstack().dropna().unique()

# Create new columns for each unique value
for value in unique_values:
    applicants_2023[f'disadvantage_{value}'] = split_columns.apply(lambda row: 1 if str(value) in row.values else 0, axis=1)
    
# List of newly created columns representing different categories extracted from the 'underrepresented' column
columns_to_convert = ['disadvantage_1', 'disadvantage_2', 'disadvantage_3','disadvantage_4','disadvantage_5',
                     'disadvantage_6','disadvantage_7','disadvantage_8'] 

# Iterate through columns_to_convert list and set the data type to 'category'
for column in columns_to_convert:
    applicants_2023[column] = applicants_2023[column].astype('category')

# Drop the original 'underrepresented' column
applicants_2023 = applicants_2023.drop(columns=['disadvantage'])

# Print the data types to confirm the changes
print(applicants_2023.dtypes)

print(applicants_2023.head(100))

selection_type            category
age                          int32
gender_id                 category
local                     category
citizenship               category
citizenship_2             category
classification            category
undergrad_major             object
minor                       object
grade_average              float64
research_opportunities    category
available                 category
summer_classes            category
concentration_choice1     category
concentration_choice2     category
concentration_choice3     category
research_experience       category
fp_choice_1               category
fp_choice_2               category
fp_choice_3               category
project_one               category
project_one_2             category
project_one_3             category
project_one_4             category
project_one_5             category
underrepresented_7        category
underrepresented_5        category
underrepresented_3        category
underrepresented_8  

In [62]:
# Split the multiple values in the 'underrepresented' column and expand into separate columns
split_columns = applicants_2023['undergrad_major'].str.split(',', expand=True)

# Get unique values from all split columns
unique_values = split_columns.unstack().dropna().unique()

# Create new columns for each unique value
for value in unique_values:
    applicants_2023[f'undergrad_major_{value}'] = split_columns.apply(lambda row: 1 if str(value) in row.values else 0, axis=1)

# List of newly created columns representing different categories extracted from the 'underrepresented' column
columns_to_convert = ['undergrad_major_1', 'undergrad_major_2', 'undergrad_major_3','undergrad_major_4','undergrad_major_5',
                     'undergrad_major_6','undergrad_major_7','undergrad_major_8','undergrad_major_9','undergrad_major_10','undergrad_major_11',
                      'undergrad_major_12','undergrad_major_13'] 

# Iterate through columns_to_convert list and set the data type to 'category'
for column in columns_to_convert:
    applicants_2023[column] = applicants_2023[column].astype('category')

# Drop the original 'underrepresented' column
applicants_2023 = applicants_2023.drop(columns=['undergrad_major'])

# Print the data types to confirm the changes
print(applicants_2023.dtypes)

print(applicants_2023.head(100))


selection_type            category
age                          int32
gender_id                 category
local                     category
citizenship               category
citizenship_2             category
classification            category
minor                       object
grade_average              float64
research_opportunities    category
available                 category
summer_classes            category
concentration_choice1     category
concentration_choice2     category
concentration_choice3     category
research_experience       category
fp_choice_1               category
fp_choice_2               category
fp_choice_3               category
project_one               category
project_one_2             category
project_one_3             category
project_one_4             category
project_one_5             category
underrepresented_7        category
underrepresented_5        category
underrepresented_3        category
underrepresented_8        category
underrepresented_6  

In [63]:
# Fill missing values in the 'minor' column with 0 (for no)
applicants_2023['minor'] = applicants_2023['minor'].fillna(0)

# Replace non-missing values with 1 (for yes)
applicants_2023['minor'] = applicants_2023['minor'].apply(lambda x: 1 if x != 0 else 0)

# Convert 'minor' column to 'category'
applicants_2023['minor'] = applicants_2023['minor'].astype('category')

# Rename the column to indicate its meaning
applicants_2023.rename(columns={'minor': 'has_minor'}, inplace=True)

# Print the data types to confirm the changes
print(applicants_2023.dtypes)

print(applicants_2023.head(100))

# Save the cleaned applicants_2023 DataFrame to a CSV file
applicants_2023.to_csv('C:/Users/ksultana/Videos/Internship/2023Applicants_clean.csv', index=False, header=True)

selection_type            category
age                          int32
gender_id                 category
local                     category
citizenship               category
citizenship_2             category
classification            category
has_minor                 category
grade_average              float64
research_opportunities    category
available                 category
summer_classes            category
concentration_choice1     category
concentration_choice2     category
concentration_choice3     category
research_experience       category
fp_choice_1               category
fp_choice_2               category
fp_choice_3               category
project_one               category
project_one_2             category
project_one_3             category
project_one_4             category
project_one_5             category
underrepresented_7        category
underrepresented_5        category
underrepresented_3        category
underrepresented_8        category
underrepresented_6  